# Introduction
Gravity waves are waves along the boundary of two mediums (e.g., atmosphere, ocean) that have a restorative gravitational force. When a displacement occurs, gravity acts to restore equilibrium along the boundary, leading to an oscillation. Shallow water gravity waves have interaction with the sea floor that leads to increased wave heights and decreased wavelengths. “Shallow” means that the depth of the fluid is very small compared to the horizontal perturbation. As a result, the waves propagate vertically and horizontally. Shallow gravity waves are important to study because they transport energy and momentum into the mid-levels of the atmosphere, leading to clear air turbulence (CAT).

In this study, the Earth will be treated as a water planet with no continents and a depth of 4km at every location. Shallow gravity waves will be along the ocean-atmosphere boundary to make use of the constants and simplifications demonstrated in Holton’s text.


# Equations
Shallow water gravity waves are horizontally propagating oscillations produced by large-scale disturbances and they can only exist in the presence of a free surface or a density discontinuity (thermocline). In this case study, the fluid has a free surface that experiences upward and downward perturbations. The perturbations create a sinusoidal pattern moving at some velocity, *u*.

We assume that the two fluids are incrompressible in order to exclude sound waves and isolate gravity waves.

The perturbation forms below are useful for this situation.

$u=\bar{u}+u^{\prime}, \quad h=H+h^{\prime}$

H is the mean depth of the lower layer and \bar{u} is a constant basic state zonal velocity. 

We are solving equation 7.22 from Holton's *An Introduction to Dynamic Meteorology* (2004): 

$$
\left(\frac{\partial}{\partial t}+u \frac{\partial}{\partial x}\right)^2 h^{\prime}-\frac{g H \delta \rho}{\rho_1} \frac{\partial^2 h^{\prime}}{\partial x^2}=0 
$$
Expansion yields can be used to cancel terms in the first parenthesis: 

$$
\begin{aligned}
& \left(\frac{\partial^2}{\partial t^2}+u^2 \frac{\partial^2}{\partial x^2}\right) h^{\prime}-\frac{g H \delta p}{\rho_1} \frac{\partial^2 h^{\prime}}{\partial x^2} \\
& \frac{\partial^2 h^1}{\partial t^2}+u^2 \frac{\partial^2 h^{\prime}}{\partial x^2}-\frac{g H \delta \rho}{\rho_1} \frac{\partial^2 h^{\prime}}{\partial x^2} \\
& \frac{\partial^2 h^{\prime}}{\partial t^2}+\left(u^2-\frac{g H \delta p}{\rho_1}\right) \frac{\partial^2 h^{\prime}}{\partial x^2}=0 \\
& \frac{\partial^2 h^{\prime}}{\partial t^2}+c \frac{\partial^2 h^{\prime}}{\partial x^2}=0 \\
\end{aligned}
$$


phase speed, $c: \quad c=u \pm \sqrt{g H}$  

Upper and lower layers are water and air. So, $\partial \rho_1 \approx \rho_1 $

$\sqrt{\mathrm{g} H} \approx 200 \mathrm{~m} / \mathrm{s}$ when $H=4 \mathrm{~km}$ as produced by large-scale disturbances, not wind

 $\frac{\partial^2 h}{\partial t^2}+c \frac{\partial^2 h}{\partial x^2}=0$

We will use this derivation to produce a simple two-dimensional model that has a flux boundary condition and a free surface. 

$$ \begin{align} u^{n+1}_i &= -u^{n-1}_i + 2u^n_i + 2C^2 \left(u^{n}_{i+1}-u^{n}_{i}\right),\quad i=0\\ u^{n+1}_i &= -u^{n-1}_i + 2u^n_i + 2C^2 \left(u^{n}_{i-1}-u^{n}_{i}\right),\quad i=N_x \end{align} $$


## Numerical Methods

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
import matplotlib
import matplotlib.animation as animation
matplotlib.use("TkAgg")

In [2]:
# parameters

xf = 24900*1000 #(m) circumference of earth at equator
xt = xf/2 #where you want the second wave speed to start
tf = 7*24*3600 #(s) 7 days in seconds
gH = 200 #(m/s) zonal wave speed in shallow water (4km)

# grid
dx = 24900
dt = 60
#dt = C*dx/c 
x = np.arange(0,xf,dx)
t = np.arange(0,tf,dt)

nx = len(x)
nt = len(t)

uv1 = 10 #average zonal velocity

# uv1 = 10 #average zonal velocity
# uv2 = -10 #average zonal velocity

# wave speed
c1 = (uv1 + gH)
c2 = (uv1 - gH)

# changing wave speed
lam = np.zeros(nx) #this is the lambda matrix
lam[x<=xt] = c1*dt/dx #sets the first half of space to have wave speed 1
lam[x>xt] = c2*dt/dx #sets the second half of space to have wave speed 2

# initial condition, ALL HAVE SAME INITIAL 
u_reflective = np.nan*np.ones([nx,nt])
u_reflective[:,0] = np.exp(-(x[:]-0)**2 / 2) ## we have to initialize both 0th and 1th because of
u_reflective[:,1] = np.exp(-(x[:]-0)**2 / 2) ## the presence of k-1 in the numerical equation. implies du/dt at t=0, =0
u_mirror = u_reflective
u_flux = u_reflective
u_periodic = u_reflective


# matrices
data1 = np.array([(lam**2)*np.ones(nx), 2*(1-lam**2)*np.ones(nx), (lam**2)*np.ones(nx)])
diags1 = np.array([-1, 0, 1])
M = spdiags(data1, diags1, nx, nx).toarray()

M_reflective = M
M_mirror = M 
M_flux = M 
M_periodic = M 

#Reflective BC
M_reflective[0,0] = 2-lam[0]**2
M_reflective[nx-1,nx-1] = 2-lam[nx-1]**2

#Mirror BC
M_mirror[nx-1, nx-2] = 0
M_mirror[nx-1,nx-1] = 1

#Flux BC

#Periodic BC 

In [3]:
# solve
for i in range(1,nt-1):
    u_reflective[:,i+1] = M_reflective @ u_reflective[:,i] - u_reflective[:,i-1]
    u_mirror[:,i+1] = M_mirror @ u_mirror[:,i] - u_mirror[:,i-1]
    
    

## Results

## Figures

In [4]:
fig, ax = plt.subplots(1,figsize = (8,5), dpi = 200)
## plotting u at all space at different times
ax.plot(x, u_reflective[:,0])
ax.plot(x, u_reflective[:,50])
ax.set_xlabel('x'); ax.set_ylabel('u')
ax.set_title("Reflective Case")
ax.legend()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [5]:
fig, ax = plt.subplots(1,figsize = (8,5), dpi = 200)
## plotting u at all space at different times
ax.plot(x, u_mirror[:,0])
ax.plot(x, u_mirror[:,50])
ax.set_xlabel('x'); ax.set_ylabel('u')
ax.set_title("Mirror Case ")
ax.legend()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


### Video Walkthrough
(insert video link)

## Reference
Holton, J. R. (2004). An introduction to dynamic meteorology. In International Geophysics (Vol. 88). https://doi.org/10.1016/s0074-6142(08)x6005-x

In [ ]:
# Reflection Animation 
animation_multiplier = 15 # Make this higher to make animation faster
fig, ax = plt.subplots()


line, = ax.plot(x, u_reflective[:,0]) #ADJUST THIS LINE
ax.set_ylim([np.min(u_reflective), np.max(u_reflective)])

def animate(i):
    line.set_ydata(u_reflective[:, i*animation_multiplier]) 
    return line,

num_frames = u_reflective.shape[1] # DO NOT TOUCH 
num_interval = 10 # DO NOT TOUCH 

ani = animation.FuncAnimation(
    fig, animate,frames=num_frames, interval=num_interval) #interval is in ms, lower means faster animation


plt.show()

In [ ]:
# Mirror  Animation 
animation_multiplier = 15 # Make this higher to make animation faster
fig, ax = plt.subplots()


line, = ax.plot(x, u_mirror[:,0]) #ADJUST THIS LINE
ax.set_ylim([np.min(u_mirror), np.max(u_mirror)])

def animate(i):
    line.set_ydata(u_mirror[:, i*animation_multiplier]) 
    return line,

num_frames = u_reflective.shape[1] # DO NOT TOUCH 
num_interval = 10 # DO NOT TOUCH 

ani = animation.FuncAnimation(
    fig, animate,frames=num_frames, interval=num_interval) #interval is in ms, lower means faster animation

plt.show()